In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Input, Dropout, BatchNormalization, Reshape, Bidirectional, Conv1D, MaxPooling1D, Flatten
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras_tuner import Hyperband, HyperParameters
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_percentage_error


import random

C:\Users\dupaw\AppData\Local\Temp\ipykernel_14456\2875046767.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Functions to modularize repeated things I want to do (loading data, custom accuracy test)

In [2]:
# Load data function
stock = {
    'AAPL': 'AAPL_1min_firstratedata.csv',
    'AMZN': 'AMZN_1min_firstratedata.csv',
    'META': 'META_1min_firstratedata.csv',
    'TSLA': 'TSLA_1min_firstratedata.csv',
    'MSFT': 'MSFT_1min_firstratedata.csv'
}
# define scalar and early stopping as global variables
scaler = MinMaxScaler(feature_range = (0, 1))

def loadData(stock = stock['AAPL'], sequence_length = 10, prediction_length = 1, want_to_plot = False): 
    """
    Loads stock data from a CSV file, applies logarithmic transformation to the volume column, 
    scales the data using a scaler, and formats it into sequences for training a model.

    Args:
        stock (str): The path to the CSV file containing the stock data. Defaults to 'AAPL'.
        sequence_length (int): The length of the input sequences. Defaults to 10.
        prediction_length (int): The length of the output sequences. Defaults to 1.
        want_to_plot (bool): Whether to plot the data. Defaults to False.

    Returns:
        tuple: A tuple containing the input sequences (x) and the output sequences (y).
    """
    # Load data
    data = pd.read_csv(stock)
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data.set_index('timestamp', inplace=True)

    if want_to_plot:
        # Plotting the data
        date = '2022-11-29 12'
        data_window = data.loc[date]
        mpf.plot(data_window, type='candle', style='charles', volume=True)
        print(data)

    # Apply logarithmic transformation to the volume column
    data['volume'] = np.log(data['volume'])
    data = pd.DataFrame(scaler.fit_transform(data), columns=data.columns, index=data.index)
    
    # e.g. for 10 timesteps per 1 prediction (sequence_length = 10)
    #  from 10 to ~160k (num observations), append data[0:10] to x for the 10 timesteps to predict data[11], the 11th prediction appended to y
    # x.append(data[1:11]) and y.append(data[12]) for the next step, etc
    x, y = [], []
    for i in range(sequence_length, len(data) - prediction_length + 1):
        x.append(data.iloc[i - sequence_length: i])
        y.append(data.iloc[i: i + prediction_length])
    x = np.array(x)
    y = np.array(y)
    return x, y

    



def calculateAccuracy(model, stock, batch = 32,want_to_print=False):
    """
    Calculates the accuracy of how often a given model predicts the correct direction on a specific stock

    Args:
        model: The model to evaluate.
        stock (str): The path to the CSV file containing the stock data.
        batch (int, optional): The batch size for prediction. Defaults to 32.
        want_to_print (bool, optional): Whether to print the results. Defaults to False.

    Returns:
        None
    """
    # Load data
    x, y = loadData(stock, sequence_length=model.input_shape[1], prediction_length=model.output_shape[1])

    # Generate predictions
    predictions = model.predict(x, batch_size=batch)

    # Check if return_sequences is True or False
    if len(predictions.shape) == 3:  # return_sequences=True
        n_predictions = predictions.shape[1]
        pred_labels = []
        true_labels = []

        for i in range(n_predictions):  # Loop through each prediction length
            pred_labels.append((predictions[:, i, 3] > x[:, -1, 3]).astype(int))
            true_labels.append((y[:, i, 3] > x[:, -1, 3]).astype(int))

        # Convert lists to numpy arrays
        pred_labels = np.array(pred_labels).T  # Transpose to match the shape
        true_labels = np.array(true_labels).T  # Transpose to match the shape

        # Calculate accuracy for each prediction length
        accuracies = []
        for i in range(n_predictions):
            accuracies.append(accuracy_score(true_labels[:, i], pred_labels[:, i]))

        # Calculate the average accuracy
        average_accuracy = np.mean(accuracies)
        print(f'Average Accuracy: {average_accuracy:.2f}')

        # Display the results
        results_dict = {
            'Previous Close': x[:, -1, 3]
        }

        for i in range(n_predictions):
            results_dict[f'Predicted Close {i+1}'] = predictions[:, i, 3]
            results_dict[f'True Close {i+1}'] = y[:, i, 3]
            results_dict[f'Predicted Direction {i+1}'] = pred_labels[:, i]
            results_dict[f'True Direction {i+1}'] = true_labels[:, i]

    else:  # return_sequences=False
        pred_labels = (predictions[:, 3] > x[:, -1, 3]).astype(int)
        true_labels = (y[:, 0, 3] > x[:, -1, 3]).astype(int)  # Adjusted to match the shape

        # Calculate accuracy
        accuracy = accuracy_score(true_labels, pred_labels)
        print(f'Accuracy: {accuracy:.2f}')

        # Display the results
        results_dict = {
            'Previous Close': x[:, -1, 3],
            'Predicted Close': predictions[:, 3],
            'True Close': y[:, 0, 3],  # Adjusted to match the shape
            'Predicted Direction': pred_labels,
            'True Direction': true_labels
        }

    results = pd.DataFrame(results_dict)
    if want_to_print:
        print(results)






Load data for this session

In [3]:

# Load data
x_aapl, y_aapl = loadData(stock=stock['AAPL'], sequence_length=40, prediction_length=1)
x_meta, y_meta = loadData(stock=stock['META'], sequence_length=40, prediction_length=1)

# Concatenate the data
x_combined = np.concatenate((x_aapl, x_meta), axis=0)
y_combined = np.concatenate((y_aapl, y_meta), axis=0)

# Split the data into training and validation sets (85% training, 15% validation)
x_train, x_val, y_train, y_val = train_test_split(x_combined, y_combined, test_size=0.15, random_state=42)
print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

(302623, 40, 5)
(53405, 40, 5)
(302623, 1, 5)
(53405, 1, 5)


Trying a bidirectional LSTM architecture: 2 LSTM, 2 Dense layers


In [5]:


# Define Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-6)
checkpoint = ModelCheckpoint('modelmk2-1.keras', monitor='val_loss', save_best_only=True, mode='min')


model = Sequential()

model.add(Input(shape=(x_train.shape[1], x_train.shape[2])))

# First Bidirectional LSTM layer with return sequences
model.add(Bidirectional(LSTM(units=160, activation='relu', return_sequences=True, kernel_regularizer=l2(6e-05))))

# Second Bidirectional LSTM layer without return sequences
model.add(Bidirectional(LSTM(units=160, activation='relu', return_sequences=False, kernel_regularizer=l2(6e-05))))

# Dense layer with 224 units
model.add(Dense(units=224, activation='relu'))
model.add(Dropout(0.10))  # Dropout layer

# Output Dense layer
model.add(Dense(units=y_train.shape[1] * y_train.shape[2], activation='relu'))
model.add(Reshape((y_train.shape[1], y_train.shape[2])))

# Compile the model with the specified learning rate
model.compile(optimizer=Adam(learning_rate=0.0006), loss='mse')
model.summary()

history = model.fit(x_train, y_train, epochs=50, batch_size=64, validation_data=(x_val, y_val), callbacks=[early_stopping, lr_scheduler, checkpoint])

# Print the model summary
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional_2 (Bidirectional) │ (None, 40, 320)        │       212,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ (None, 320)            │       615,680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 224)            │        71,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 224)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 5)              │         1,125 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_1 (Reshape)             │ (None, 1, 5)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 901,189 (3.44 MB)

 Trainable params: 901,189 (3.44 MB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional_2 (Bidirectional) │ (None, 40, 320)        │       212,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ (None, 320)            │       615,680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 224)            │        71,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 224)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 5)              │         1,125 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_1 (Reshape)             │ (None, 1, 5)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,703,569 (10.31 MB)

 Trainable params: 901,189 (3.44 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,802,380 (6.88 MB)

Trying a CNN-LSTM architecture: 2 conv1d -> max pooling -> 2LSTM2Dense

In [7]:

# Define Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-6)
checkpoint = ModelCheckpoint('modelmk2-2.keras', monitor='val_loss', save_best_only=True, mode='min')


model = Sequential()

model.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
# model.add(Dropout(0.2))
# First Bidirectional LSTM layer with return sequences
model.add(LSTM(units=160, activation='relu', return_sequences=True, kernel_regularizer=l2(6e-05)))
# Second Bidirectional LSTM layer without return sequences
model.add(LSTM(units=160, activation='relu', return_sequences=False, kernel_regularizer=l2(6e-05)))

# Dense layer with 224 units
model.add(Dense(units=224, activation='relu'))
# model.add(Dropout(0.15))  # Dropout layer

# Output Dense layer
model.add(Dense(units=y_train.shape[1] * y_train.shape[2], activation='relu'))
model.add(Reshape((y_train.shape[1], y_train.shape[2])))
model.summary()
# Compile the model with the specified learning rate
model.compile(optimizer=Adam(learning_rate=0.0006), loss='mse')
history = model.fit(x_train, y_train, epochs=50, batch_size=64, validation_data=(x_val, y_val), callbacks=[early_stopping, lr_scheduler, checkpoint])

# Print the model summary


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)               │ (None, 38, 64)         │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 36, 64)         │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 18, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 18, 160)        │       144,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 160)            │       205,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 224)            │        36,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 5)              │         1,125 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_2 (Reshape)             │ (None, 1, 5)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 400,005 (1.53 MB)

 Trainable params: 400,005 (1.53 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
4729/4729 ━━━━━━━━━━━━━━━━━━━━ 185s 38ms/step - loss: 0.0084 - val_loss: 0.0012 - learning_rate: 6.0000e-04
Epoch 2/50
4729/4729 ━━━━━━━━━━━━━━━━━━━━ 178s 38ms/step - loss: 0.0012 - val_loss: 0.0011 - learning_rate: 6.0000e-04
Epoch 3/50
4729/4729 ━━━━━━━━━━━━━━━━━━━━ 172s 36ms/step - loss: 0.0011 - val_loss: 0.0010 - learning_rate: 6.0000e-04
Epoch 4/50
4729/4729 ━━━━━━━━━━━━━━━━━━━━ 177s 37ms/step - loss: 0.0011 - val_loss: 0.0010 - learning_rate: 6.0000e-04
Epoch 5/50
4729/4729 ━━━━━━━━━━━━━━━━━━━━ 175s 37ms/step - loss: 0.0010 - val_loss: 0.0011 - learning_rate: 6.0000e-04
Epoch 6/50
4729/4729 ━━━━━━━━━━━━━━━━━━━━ 174s 37ms/step - loss: 0.0010 - val_loss: 0.0010 - learning_rate: 6.0000e-04
Epoch 7/50
4729/4729 ━━━━━━━━━━━━━━━━━━━━ 175s 37ms/step - loss: 0.0010 - val_loss: 0.0010 - learning_rate: 3.0000e-04
Epoch 8/50
4729/4729 ━━━━━━━━━━━━━━━━━━━━ 174s 37ms/step - loss: 0.0010 - val_loss: 9.8175e-04 - learning_rate: 3.0000e-04
Epoch 9/50
4729/4729 ━━━━━━━━━━━━━━━━━━━━ 17

Testing MSE, MAPE, and custom accuracy:

In [8]:
model = Sequential()
model = load_model('modelmk2-2.keras')


x, y = loadData(stock=stock['TSLA'], sequence_length=40, prediction_length=1)
# Make predictions on the validation set
y_val_pred = model.predict(x)
# Reshape the arrays to be 2-dimensional
y = y.reshape(-1, y.shape[-1])
y_val_pred = y_val_pred.reshape(-1, y_val_pred.shape[-1])


# Inverse transform the predictions and actual values
y_val_pred_unscaled = scaler.inverse_transform(y_val_pred)
y_unscaled = scaler.inverse_transform(y)
# Calculate MSE on the unscaled data
mse_unscaled = mean_squared_error(y_unscaled, y_val_pred_unscaled)
print(f'Mean Squared Error (Unscaled): {mse_unscaled}')

# Calculate MAPE on the unscaled data
mape_unscaled = mean_absolute_percentage_error(y_unscaled, y_val_pred_unscaled)
print(f'Mean Absolute Percentage Error (Unscaled): {mape_unscaled}')

7212/7212 ━━━━━━━━━━━━━━━━━━━━ 68s 9ms/step
Mean Squared Error (Unscaled): 0.5307263691644964
Mean Absolute Percentage Error (Unscaled): 0.016029600633783773


In [9]:
calculateAccuracy(model, stock['AAPL'], 64)
calculateAccuracy(model, stock['AMZN'], 64)
calculateAccuracy(model, stock['META'], 64)
calculateAccuracy(model, stock['MSFT'], 64)
calculateAccuracy(model, stock['TSLA'], 64)

3028/3028 ━━━━━━━━━━━━━━━━━━━━ 41s 14ms/step
Average Accuracy: 0.51
3025/3025 ━━━━━━━━━━━━━━━━━━━━ 43s 14ms/step
Average Accuracy: 0.50
2535/2535 ━━━━━━━━━━━━━━━━━━━━ 35s 14ms/step
Average Accuracy: 0.52
2484/2484 ━━━━━━━━━━━━━━━━━━━━ 34s 14ms/step
Average Accuracy: 0.52
3606/3606 ━━━━━━━━━━━━━━━━━━━━ 49s 14ms/step
Average Accuracy: 0.52


Adding more units and filters to the model:

In [12]:

# Define Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-6)
checkpoint = ModelCheckpoint('modelmk2-3.keras', monitor='val_loss', save_best_only=True, mode='min')


model = Sequential()

model.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
# model.add(Dropout(0.2))
# First  LSTM layer with return sequences
model.add(LSTM(units=260, activation='relu', return_sequences=True, kernel_regularizer=l2(6e-05)))
# Second  LSTM layer without return sequences
model.add(LSTM(units=260, activation='relu', return_sequences=False, kernel_regularizer=l2(6e-05)))

# Dense layer with 224 units
model.add(Dense(units=224, activation='relu'))
# model.add(Dropout(0.15))  # Dropout layer

# Output Dense layer
model.add(Dense(units=y_train.shape[1] * y_train.shape[2], activation='relu'))
model.add(Reshape((y_train.shape[1], y_train.shape[2])))
model.summary()
# Compile the model with the specified learning rate
model.compile(optimizer=Adam(learning_rate=0.0006), loss='mse')
history = model.fit(x_train, y_train, epochs=50, batch_size=64, validation_data=(x_val, y_val), callbacks=[early_stopping, lr_scheduler, checkpoint])

# Print the model summary


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_8 (Conv1D)               │ (None, 38, 128)        │         2,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_9 (Conv1D)               │ (None, 36, 128)        │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_4 (MaxPooling1D)  │ (None, 18, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_10 (LSTM)                  │ (None, 18, 260)        │       404,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_11 (LSTM)                  │ (None, 260)            │       541,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 224)            │        58,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 5)              │         1,125 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_5 (Reshape)             │ (None, 1, 5)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,057,317 (4.03 MB)

 Trainable params: 1,057,317 (4.03 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
4729/4729 ━━━━━━━━━━━━━━━━━━━━ 325s 68ms/step - loss: 0.0230 - val_loss: 0.0011 - learning_rate: 6.0000e-04
Epoch 2/50
4729/4729 ━━━━━━━━━━━━━━━━━━━━ 320s 68ms/step - loss: 0.0012 - val_loss: 0.0011 - learning_rate: 6.0000e-04
Epoch 3/50
4729/4729 ━━━━━━━━━━━━━━━━━━━━ 320s 68ms/step - loss: 0.0011 - val_loss: 0.0010 - learning_rate: 6.0000e-04
Epoch 4/50
4729/4729 ━━━━━━━━━━━━━━━━━━━━ 319s 67ms/step - loss: 0.0011 - val_loss: 9.9914e-04 - learning_rate: 6.0000e-04
Epoch 5/50
4729/4729 ━━━━━━━━━━━━━━━━━━━━ 330s 70ms/step - loss: 0.0010 - val_loss: 0.0010 - learning_rate: 6.0000e-04
Epoch 6/50
4729/4729 ━━━━━━━━━━━━━━━━━━━━ 333s 70ms/step - loss: 0.0010 - val_loss: 0.0010 - learning_rate: 6.0000e-04
Epoch 7/50
4729/4729 ━━━━━━━━━━━━━━━━━━━━ 332s 70ms/step - loss: 0.0010 - val_loss: 9.8795e-04 - learning_rate: 6.0000e-04
Epoch 8/50
4729/4729 ━━━━━━━━━━━━━━━━━━━━ 325s 69ms/step - loss: 0.0010 - val_loss: 9.8560e-04 - learning_rate: 6.0000e-04
Epoch 9/50
4729/4729 ━━━━━━━━━━━━━━━

Testing model efficacy:


In [16]:
model = Sequential()
model = load_model('modelmk2-3.keras')


x, y = loadData(stock=stock['TSLA'], sequence_length=40, prediction_length=1)
# Make predictions on the validation set
y_val_pred = model.predict(x)
# Reshape the arrays to be 2-dimensional
y = y.reshape(-1, y.shape[-1])
y_val_pred = y_val_pred.reshape(-1, y_val_pred.shape[-1])


# Inverse transform the predictions and actual values
y_val_pred_unscaled = scaler.inverse_transform(y_val_pred)
y_unscaled = scaler.inverse_transform(y)
# Calculate MSE on the unscaled data
mse_unscaled = mean_squared_error(y_unscaled, y_val_pred_unscaled)
print(f'Mean Squared Error (Unscaled): {mse_unscaled}')

# Calculate MAPE on the unscaled data
mape_unscaled = mean_absolute_percentage_error(y_unscaled, y_val_pred_unscaled)
print(f'Mean Absolute Percentage Error (Unscaled): {mape_unscaled}')

7212/7212 ━━━━━━━━━━━━━━━━━━━━ 98s 13ms/step
Mean Squared Error (Unscaled): 0.5081611660577526
Mean Absolute Percentage Error (Unscaled): 0.015934510413404154


In [17]:
calculateAccuracy(model, stock['AAPL'], 64)
calculateAccuracy(model, stock['AMZN'], 64)
calculateAccuracy(model, stock['META'], 64)
calculateAccuracy(model, stock['MSFT'], 64)
calculateAccuracy(model, stock['TSLA'], 64)

3028/3028 ━━━━━━━━━━━━━━━━━━━━ 59s 19ms/step
Average Accuracy: 0.51
3025/3025 ━━━━━━━━━━━━━━━━━━━━ 60s 20ms/step
Average Accuracy: 0.50
2535/2535 ━━━━━━━━━━━━━━━━━━━━ 50s 20ms/step
Average Accuracy: 0.51
2484/2484 ━━━━━━━━━━━━━━━━━━━━ 50s 20ms/step
Average Accuracy: 0.52
3606/3606 ━━━━━━━━━━━━━━━━━━━━ 71s 20ms/step
Average Accuracy: 0.52


Running the Hyperband hyperparameter tuner to find the best model architecture: 1-3 Conv1d layers tuning number of filters, 1-3 LSTM layers tuning kernal regularization and number of units, 0-2 dense layers tuning the number of units, and then also tuning learning rate: 

In [4]:
# Define Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-6)
checkpoint = ModelCheckpoint('modelmk2-4.keras', monitor='val_loss', save_best_only=True, mode='min')

def build_model(hp):
    model = Sequential()
    model.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
    
    # Conv1D layers
    for i in range(hp.Int('conv_layers', 1, 3)):
        model.add(Conv1D(filters=hp.Int(f'filters_{i}', 32, 256, step=32),
                         kernel_size=3, activation='relu'))
    
    model.add(MaxPooling1D(pool_size=2))
    
    # LSTM layers
    num_lstm_layers = hp.Int('lstm_layers', 1, 3)
    for i in range(num_lstm_layers):
        model.add(LSTM(units=hp.Int(f'lstm_units_{i}', 64, 512, step=64),
                    activation='relu', return_sequences=(i < num_lstm_layers - 1),
                    kernel_regularizer=l2(hp.Float(f'l2_{i}', 1e-6, 1e-4, sampling='LOG'))))

    
    # Dense layers
    for i in range(hp.Int('dense_layers', 0, 2)):
        model.add(Dense(units=hp.Int(f'dense_units_{i}', 64, 512, step=64), activation='relu'))
    
    # Output layer
    model.add(Dense(units=y_train.shape[1] * y_train.shape[2], activation='relu'))
    model.add(Reshape((y_train.shape[1], y_train.shape[2])))
    
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=hp.Float('learning_rate', 1e-5, 1e-3, sampling='LOG')),
                  loss='mse')
    return model

tuner = Hyperband(
    build_model,
    objective='val_loss',
    max_epochs=30,
    factor=3,
    hyperband_iterations=1,
    directory=r"C:\Users\dupaw\Desktop\projects\algotrading",
    project_name='stock_prediction_pt2'
)

tuner.search(x_train, y_train, epochs=30, validation_data=(x_val, y_val), callbacks=[early_stopping, lr_scheduler, checkpoint], batch_size=64)

# # Compile the model with the specified learning rate
# model.compile(optimizer=Adam(learning_rate=0.0003), loss='mse')


Trial 69 Complete [00h 46m 06s]
val_loss: 0.0009449672070331872

Best val_loss So Far: 0.0008965398883447051
Total elapsed time: 2d 13h 04m 23s

Search: Running Trial #70

Value             |Best Value So Far |Hyperparameter
3                 |1                 |conv_layers
256               |64                |filters_0
2                 |1                 |lstm_layers
512               |128               |lstm_units_0
7.8472e-06        |1.0572e-06        |l2_0
1                 |2                 |dense_layers
0.0002858         |0.00080814        |learning_rate
256               |192               |filters_1
384               |64                |lstm_units_1
4.5748e-05        |3.3546e-06        |l2_1
448               |128               |lstm_units_2
4.6963e-05        |5.533e-06         |l2_2
128               |448               |dense_units_0
128               |192               |filters_2
384               |64                |dense_units_1
10                |30                |tune

C:\Users\dupaw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 34 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


Epoch 5/10
 280/4729 ━━━━━━━━━━━━━━━━━━━━ 11:23 154ms/step - loss: 0.0013